In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

## 3D meshing example

This notebook shows how to mesh a 3D volume:

1. Load and visualize a volume
1. Apply image filters and segment image
1. Generate a 3D surface mesh from the binary image
1. Decimate/simplify a 3D mesh
1. Visualize and save the mesh to gmsh22 format

## Todo:

- [ ] Mimic `add_points` from 2d meshing submodule
- [ ] Tune performance
- [ ] Add edge points instead of padding?

In [ ]:
from nanomesh.volume import Volume
import pyvista as pv
from skimage import filters
import numpy as np

Load the data and select a subvolume to work with for this notebook.

In [ ]:
full_vol = Volume.load('slab_x450-550.npy')
vol = full_vol.select_subvolume(zs=(700, 800), ys=(300,400))
vol.show_slice()

In [ ]:
vol_gauss = vol.apply(filters.gaussian, sigma=5)

thresh = vol_gauss.apply(filters.threshold_li)

seg_vol = vol_gauss.apply(np.digitize, bins=[thresh])

seg_vol.show_slice()

### Generate 3d tetragonal mesh

Meshes can be generated using the `Mesher3D` class.

A convenience method is also available through `volume.generate_mesh`, which offers less flexibility.

In [ ]:
%%time

from nanomesh.mesh3d import Mesher3D

mesher = Mesher3D(seg_vol.image)
mesher.pad(pad_width=2)
mesher.add_points(point_density=1/500, step_size=1, label=1, verbose=3)
mesher.generate_surface_mesh(step_size=2)
mesher.smooth_mesh()
mesher.simplify_mesh(n_faces=5000)
mesher.generate_volume_mesh()
mesher.generate_domain_mask()
mesh = mesher.to_meshio()

pv.plot_itk(mesh)

Show subsection of the tetragonal mesh.

In [ ]:
from nanomesh.mesh3d import show_submesh

show_submesh(mesh, index=30)

Save the data.

In [ ]:
mesh.write("volume.msh", file_format='gmsh22', binary=False)